# Introduction à Flask


## Introduction

**Flask** est, avec Django, l'une des plates-formes (ou « framework ») web les plus populaires de l'écosystème Python. Elle a été conçue pour être très légère.

### Que signifie "micro" ?

Le "micro" dans microframework signifie que Flask vise à garder le noyau simple mais extensible. Flask ne prendra pas beaucoup de décisions pour vous, comme le choix de la base de données à utiliser. Les décisions qu'il prend, comme le moteur de templating à utiliser, sont faciles à modifier. Tout le reste dépend de vous, de sorte que Flask puisse être tout ce dont vous avez besoin et rien que vous n'ayez besoin.

Par défaut, Flask n'inclut pas de couche d'abstraction de base de données, de validation de formulaire ou quoi que ce soit d'autre pour lequel il existe déjà des bibliothèques différentes qui peuvent s'en charger. Au lieu de cela, Flask supporte des extensions pour ajouter de telles fonctionnalités à votre application comme si elles étaient implémentées dans Flask lui-même.

### Configuration et conventions

Flask possède de nombreuses valeurs de configuration, avec des valeurs par défaut raisonnables, et quelques conventions pour commencer. Par convention, les fichiers templates et statiques sont stockés dans des sous-répertoires de l'arbre source Python de l'application, avec les noms `templates` et `static` respectivement. Bien que cela puisse être modifié, vous n'avez généralement pas besoin de le faire, surtout au début.

### Évolution avec Flask

Une fois que vous avez mis en place Flask, vous trouverez une variété d'extensions disponibles dans la communauté pour intégrer votre projet à la production.

À mesure que votre base de code se développe, vous êtes libre de prendre les décisions de conception appropriées à votre projet. Flask continuera à fournir une couche de colle très simple à ce que Python a de mieux à offrir. Vous pouvez mettre en œuvre des modèles avancés dans SQLAlchemy ou un autre outil de base de données, introduire la persistance des données non relationnelles, le cas échéant, et tirer parti des outils agnostiques construits pour WSGI, l'interface web Python.

Flask comprend de nombreux hooks permettant de personnaliser son comportement. Si vous avez besoin de plus de personnalisation, la classe Flask est conçue pour être sous-classée. Si cela vous intéresse, consultez le chapitre " Becoming Big ". Si vous êtes curieux de connaître les principes de conception de Flask, rendez-vous à la section sur les décisions de conception dans Flask.

## Premiers pas

### Installation

La plate-forme s'installe très facilement avec pip, comme d'habitude en Python :

```bash
pip install flask
```

Le cas échéant, vous pouvez bien sûr créer un environnement virtuel pour encapsuler votre application.

> **Rappel : Création d'environnement virtuel**
>
> Un environnement virtuel est un espace défini pour une application Python, isolé du reste du système, et qui contient ses propres modules, et éventuellement sa propre version de Python.
> Pour créer un environnement virtuel :
>```bash
> python -m venv <nom_de_dossier>
>```
> L'environnement virtuel doitensuite être activé pour entrer en service :
>```bash
> source <nom_de_dossier>/bin/activate
>```
> Vous pouvez enfin le désactiver en fin de session :
>```bash
> deactivate
>```

Flask installe elle-même quelques bibliothèques :

- **Werkzeug**, pour WSGI, l'interface entre Python et les serveurs web
- **Jinja**, un moteur de templates HTML très semblable à Twig
- **MarkupSafe**, une extension de Jinja destinée à éviter les attaques par injection en protégeant les chaînes de caractères
- **ItsDangerous**, un module pour signer les données (par exemple les cookies de session de Flask).
- **Click**, un outil pour la gestion des applications Flask en ligne de commande.

Une fois installée, Flask n'est pas immédiatement opérationnel. Nous allons devoir ajouter quelques fichiers pourra afficher une page. D'ailleurs, si vous avez installé Flask au niveau « global », vous pouvez créer un dossier n'importe où dans votre système de fichiers et commencer à écrire le code.

### Publication sur le web

Contrairement à PHP, qui est une exception, Python n'est pas directement reconnu par les serveurs HTTP, en particulier par Apache. Il existe globalement trois solutions possibles :

1. Flask possède son propre démon HTTP, il suffira de le lancer, comme nous le verrons et l'application sera directement disponible. C'est une solution qui offre l'avantge de la simplicité, mais qui est très peu efficace. Néanmoins, en phase de développement ou d'expérimentation, elle peut se concevoir.
2. Si vous utilisez le serveur HTTP **Nginx**, vous pouvez installez **gunicorn**, qui est plus puissant que le serveur interne de Flask et le lier ensuite à Nginx qui assurera la publication sur le web.
3. Si vous utilisez **Apache**, la procédure est plus complexe ; vous devrez installer le module `mod_wsgi` de Python ainsi que l'extension équivalente pour Apache. Cela demandera un pett peu pllus de travail.

Par la suite, nous admetrons tout simplement qu'une de ces solutions sont actives.

### Structure de l'application

Au départ, le dossier de votre application est vide, à moins que vous n'ayez opté pour un environnement virtuel.

L'application Flask minimale ne comprte qu'un seul fichier, que vous baptisez du nom que vous voulez. Appelon-le `server.py`.

In [ ]:
# server.py

# a minima import l'objet Flask, qui est le nnoyau de l'application
from flask import Flask

# Créer l'application (rappel : `__name__` contient le nom du fichier dans lequel elle est appelée)
app = Flask(__name__)

# Lancement de l'application
# Le mode 'debug' permet de redémarrer le serveur automatiquement lorsque le code est modifié 
# et d'afficher une trace dans le navigateur en cas d'erreur
# Vous pouvez spécifier le port sur lequel Flask écoutera les requêtes, 5000 est le port par défaut
if __name__ == '__main__' :
    app.run(debug=True, port=5000)

Pour mettre en route le serveur avec le démon de Flask, il ssuffit d'exécuter la commande shell :
```bash
FLASK_APP=server.py flask run
```
> **N.B.** Notez que si votre fichier s'appelle `app.py` ou `wsgi.py`, la variable système est inutile.

Si vous exécutez directement le script, vous allez certainement constater qu' il ne se passe rien ! En effet, si le moteur est actif, il manque la chose principale : la description de l'API de l'application, c'est-à-dire l'ensemble des routes accessibles par les utilisateurs.

Chaque route comprend deux faces:
- un décorateur qui définit la structure de l'URL
- une fonction qui traite la requête, appelée « contrôleur » (ou quelquefois « adapteur »)

In [ ]:
# Route minimale pour la page d'accueil
# Le décorateur `app.route` est défini dans l'objet `app` qui représente l'application elle-même

@app.route('/')
def home():
    return "<h1>Accueil<h1><p>Première page</p>"

@app.route('/about-us')
def about():
    return "<h1>Voilà !<h1><p>C'est moi qui l'ai fait</p>"
    

Vous pouvez définir autant de routes que vous le souhaitez.

Normalement, vous devriez pouvoir maintnant afficher quelque chose dans le navigateur à l'adresse `http://localhost:5000/about_us`

#### Autre dossiers minimaux

Comme nous venons de le voir l'application peut se résumer à un script. Néanmoins, dans un cas réel, nous aurons beson d'au moins deux dossiers, à la racine de l'application :
- `static`, qui contiendra toutes les ressources web
- `templates`, qui contiendra les gabarits d'affichage des pages, écrites en HTMl ou avec Jinja.

Par exemple, si vous voulez afficher une page statique de présentation de l'application, vous pourrez écrire :

In [ ]:
# import de la fonction `render_template`
from flask import render_template

@app.route('/about-us')
def about():
    # Le gabarit `about_us.html` est par défaut dans le dossier `templates`
    return render_template('about_us.html')

### Routes

En dehors de leur déclaration minimale, les routes peuvent accepter un  certain nombre d'options

#### Variables de routes

Une route définit en faut un _schéma_ d'URL, qui peut comprendre des variables. Par exemple :

In [ ]:
# La variable `id` est signalée par les chevrons
@app.route('/model/<id>')
# La variable est également introduite comme paramètre de la fonction
def about(id):
    # Elle peut ensuite être passée au gabarit pour être affichée
    return render_template('model.html', id=id)

#### Méthodes HTTP

Lors de la définition des API, la bonne pratique est d'utiliser complètement les méthodes HTTP, ce qui permet en particulier de définir moins de schémas d'URL. On peut donc tout à fait écrire :

In [ ]:
# Une route pour afficher la page d'un objet
@app.route('/model/<id>', methods=['GET'])
def about(id):
    return render_template('model.html', id=id)

# Une autre route avec le smême schéma pour modifier les données de cet objet
@app.route('/model/<id>', methods=['PATCH'])
def about(id):
    # `update_model` est une fonction fictive :o)
    update_model(id)
    return render_template('model.html', id=id)

### Requêtes

Les paramètres de la requête HTTP  sont disponibles sou la forme de la variable globale `request`. Exemple :

In [ ]:
# Existence de valeurs dan la chaîne de requête de l'URL = `/model/25?version=1:50

@app.route('/model/<id>', methods=['GET'])
def about(id):
    # la propriété `args` de l'objet `request` fournit la valeur demandée
    version = request.args.get('version', '1.10')
    return render_template('model.html', id=id, version=version)


Tous les éléments de la requêtes (cookies, formulaires (body), fichiers, entêtes HTTP, etc.) sont naturellement disponibles de la même facçon.

### Réponses

Flask fournit par défaut des réponses au format HTML, c'est le rôle de la fonctin `render_template`.
A noter que le code de statut de la réponse peut être aisément ajouté :

In [ ]:
@app.route('/model/<id>', methods=['GET'])
def about(id):
    version = request.args.get('version', '1.10')
    # Le code 200 est précisé (de manière redondante, dans ce cas)
    return render_template('model.html', id=id, version=version), 200


Il existe d'autres méthodes pour définir les réponses.

#### make_response

`make_response` se contente de construire l'objet. L'avantage de cette solution est que vous pouvez ajouter par la suite tous les entêtes HTTP que vous voulez.

In [ ]:
@app.route('/model/<id>', methods=['GET'])
def about(id):
    version = request.args.get('version', '1.10')
    response = make_response('model.html', id=id, version=version), 200
    # Ajout d'entêtes
    resp.headers['Last-Modified'] = 'Sat, 20 Nov 2021 16:28:55 GMT'
    resp.headers['Etag'] = '2c78a220b1e9b81779410856d2cb59cb44f928b9'
    # Envoi de la réponse
    return response


#### json

Dans le cas d'un serveur d'API, vous pourrez facilement envoyer des réponses au format JSON :

In [ ]:
# Directement
@app.route("/me")
def me_api():
    user = get_current_user()
    return {
        "username": user.username,
        "theme": user.theme,
        "image": url_for("user_image", filename=user.image),
    }

# En linéarisant les données
from flask import jsonify

@app.route("/users")
def users_api():
    users = get_all_users()
    return jsonify([user.to_json() for user in users])


En pratique, la linéarisation se fera plus souvent via des bibliothèques comme `pickle` ou `marshmallow`.

### Services

Enfin, nous avons surtout vu des exemples très simples de traitement des requêtes, mais les contrôleurs peuvent contenir tout code arbitraire qui trnasforme les données d'entrrée pour fournir la réponse.

La bonne pratique est de déporter tous ces traitements dans de modules externes,qu'on appellera services. PAr exemple ;

In [ ]:
# En linéarisant les données
from flask import jsonify
import user

@app.route("/users")
def users_api():
    # une fonction `all` d'un module `user` est préférable à la fonction globale `get_all_users`
    users = user.all()
    return jsonify([user.to_json() for user in users])
